In [ ]:
years = [2022]

In [ ]:
dates_list = []

for year in years:
    for month in range(1, 13):
        days = None
        if month in [1,3,5,7,8,10,12]:
            days = 31
        elif month == 2 and year != 2020:
            days = 28
        elif month == 2 and year == 2020:
            days = 29
        else:
            days = 30
        for i in range(1, days + 1):
            formatted_month = ''
            if month < 10:
                formatted_month = '0' + str(month)
            else:
                formatted_month = str(month)
            formatted_day = '0'
            if i < 10:
                formatted_day = '0' + str(i)
            else:
                formatted_day = str(i)
            date = str(year) + '-' + formatted_month + '-' + formatted_day 
            dates_list.append(date)

Almost full list of Airport Codes (IATA) can be found here:

https://www.nationsonline.org/oneworld/IATA_Codes/airport_code_list.htm

Put codes you are interested in in airports_list, the code will requiest all possible combinations

In [ ]:
airports_list = []
departure_airports = ["DOH"]

Airport codes were taken here: https://www.id1.de/2021/08/25/middle-east-africa-top-50-largest-airports-in-2021-by-number-of-flights/

In [ ]:
f = open("airports.txt")
for line in f:
    airports_list.append(line.rstrip())
f.close()

In [ ]:
import psycopg2

db_connection = psycopg2.connect(
            host="localhost",
            user="user",
            password=password,
            database="database"
        )
cursor = db_connection.cursor()

# in case you need to clear data in the table

# query = "DELETE FROM tickets_research"
# cursor.execute(query)
# db_connection.commit()

In [ ]:
from amadeus import Client, ResponseError

keys = [
    ["client_id" , "client_secret"],
]

In [ ]:
def add_to_df(data):
    n = len(data)
    for i in range(n):
        segments = data[i]["itineraries"][0]["segments"]
        Departure_Date, Departure_time = map(str, segments[0]["departure"]["at"].split("T"))
        Arrival_date, Arrival_time = map(str, segments[len(segments) - 1]["arrival"]["at"].split("T"))
        Departure_airport = segments[0]["departure"]["iataCode"]
        Destination_airport = segments[len(segments) - 1]["arrival"]["iataCode"]
        One_Way = data[i]["oneWay"]
        Flight_Time = data[i]["itineraries"][0]["duration"][2:]
        Stops_number = len(segments) - 1
        Stops_list = []
        for segment in segments[:1]:
            Stops_list.append(segment["arrival"]["iataCode"])
        Flight_class = data[i]["travelerPricings"][0]["fareDetailsBySegment"][0]["cabin"]
        Total_price = data[i]["price"]["total"]
        Base_price = data[i]["price"]["base"]
        Additional_Services = 0
        try:
            for j in range(len(data[i]["price"]["additionalServices"])):
                Additional_Services += float(data[i]["price"]["additionalServices"][i]["amount"])
            Additional_Services = str(Additional_Services)
        except:
            Additional_Services = "None"
        Currency = data[i]["price"]["currency"]
        Carrier_Codes = data[i]["validatingAirlineCodes"][0]
        query = "INSERT INTO tickets_research(Departure_Date,Departure_airport,Destination_airport,\
                    Departure_time, One_Way,Arrival_time,Flight_Time,Stops_number,Stops_list,Flight_class,\
                    Total_price,Base_price,Currency,Carrier_Codes) \
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        cursor.execute(query, [Departure_Date,Departure_airport,Destination_airport,Departure_time,
                       One_Way,Arrival_time,Flight_Time,Stops_number,Stops_list,Flight_class,Total_price,
                       Base_price,Currency,Carrier_Codes])
        db_connection.commit()

Departure_Date | Departure_airport | Destanation_airport | Departure_time | One_Way | Arrival_time | Flight_Time | Stops_number | Stops_list |Flight_class | Total_price | Base_price | Currency | Carrier_Codes

In [ ]:
keys_count = 0
amadeus = Client(
    client_id=keys[keys_count][0],
    client_secret=keys[keys_count][1]
)
count = 0
for date in dates_list:
    for departure in departure_airports:
        for arrival in airports_list:
            try:
                response = amadeus.shopping.flight_offers_search.get(
                    originLocationCode=departure,
                    destinationLocationCode=arrival,
                    departureDate=date,
                    adults=1)
                add_to_df(response.result["data"])
            except ResponseError as error:
                print("Error", error, "Date", date, "Departure=", departure, "Destination=", arrival)
            try:
                response = amadeus.shopping.flight_offers_search.get(
                    originLocationCode=arrival,
                    destinationLocationCode=departure,
                    departureDate=date,
                    adults=1)
                add_to_df(response.result["data"])
            except ResponseError as error:
                print("Error", error, "Date", date, "Departure=", departure, "Destination=", arrival)
            count += 2
            if count == 2000:
                count = 0
                keys_count += 1
                amadeus = Client(
                    client_id=keys[keys_count][0],
                    client_secret=keys[keys_count][1]
                )
    